In [1]:
import numpy as np
import pandas as pd

In [2]:
with open('./eval.txt', 'r') as f:
    results = f.readlines()

# Remove any non-result lines from the eval file, and split the lines on the tab character
# (results have format: model_name\tdataset_name\tmetric_name\tmetric_value)
results = [r.replace('\n','').split('\t') for r in results if '\t' in r]

In [3]:
def remove_underscore_after(val): # Remove underscore, and keep the part after the underscore
    return val.split('_')[-1]
def remove_underscore_before(val): # Remove underscore, and keep the part before the underscore
    return val.split('_')[0]
def map2d(func, grid): # Mapping for 2d arrays, from: https://stackoverflow.com/questions/70742445/elegant-map-over-2d-list
    return [[func(value) for value in row] for row in grid]
def full_display(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        display(df)

df = pd.DataFrame(results, columns = ['model', 'dataset', 'metric', 'value'])
df['value'] = pd.to_numeric(df['value'])
df['model'] = df['model'].map(lambda x: '-'.join(x.split('-')[2:]))
models = df['model'].str.split('-').tolist()

# Remove all underscores from our 2d list, keep one list of the param names and one with param vals
model_names_list = map2d(remove_underscore_before, models)
model_names = np.array(model_names_list)
model_val_list = map2d(remove_underscore_after, models)
model_vals = np.array(model_val_list)

In [4]:
model_vals.shape, model_names.shape

((36030, 14), (36030, 14))

In [5]:
# Splitting model name into more columns
for i in range(model_names.shape[-1]):
    name = model_names[0][i]
    val = model_vals[:,i]
    df[name] = val
    try:
       df[name] = pd.to_numeric(df[name]) 
    except:
        pass

# Remove columns that aren't relevant
df = df.drop(['fold', 'vit', 'model', 'data', 'ALL',  'kw'], axis = 1) #'method', 'AL.iter', 'ratio', 'PL',

# Replace 'None' with NaN, to allow conversion to numerical
df['AL.iter'] = df['AL.iter'].replace('None', np.nan)
df['AL.iter'] = pd.to_numeric(df['AL.iter'])
df['AL.epochs'] = df['AL.epochs'].replace('None', np.nan)
df['AL.epochs'] = pd.to_numeric(df['AL.epochs'])

cols = sorted(df.columns.tolist())
print(cols)
# The ratio have the format: 'ratio_xxx' where xxx is some float.
# We only want that float value, and need to convert it from string to float.
# df['ratio'] = pd.to_numeric(df['ratio'])
display(df)

# Group by all columns except the performance values, then compute mean, std and number of model runs for the performance
df_grouped = df.groupby(list(set(cols)-set(['value'])), dropna = False).agg({'value':['mean', 'std', 'count']})

['AL.epochs', 'AL.iter', 'PL', 'bs', 'dataset', 'epochs', 'lr', 'method', 'metric', 'ratio', 'value']


,dataset,metric,value,ratio,method,AL.iter,AL.epochs,PL,epochs,lr,bs
0,RSICD-CLS,zeroshot-val-top1,60.99,0.1,base,NaN,NaN,None,5,0.000050,64
1,UCM-CLS,zeroshot-val-top1,56.84,0.1,base,NaN,NaN,None,5,0.000050,64
2,RSICD,image_to_text_R@1,4.76,0.1,base,NaN,NaN,None,5,0.000050,64
3,RSICD,image_to_text_R@5,20.05,0.1,base,NaN,NaN,None,5,0.000050,64
4,RSICD,image_to_text_R@10,33.88,0.1,base,NaN,NaN,None,5,0.000050,64
...,...,...,...,...,...,...,...,...,...,...,...
36025,Sydney,image_to_text_R@5,11.87,0.1,ours,NaN,NaN,hard.text,35,0.000005,128
36026,Sydney,image_to_text_R@10,20.12,0.1,ours,NaN,NaN,hard.text,35,0.000005,128
36027,Sydney,text_to_image_R@1,3.62,0.1,ours,NaN,NaN,hard.text,35,0.000005,128
36028,Sydney,text_to_image_R@5,12.88,0.1,ours,NaN,NaN,hard.text,35,0.000005,128


In [6]:
display(df_grouped)

value  \
                                                                                                mean   
lr       ratio method AL.epochs bs  PL        epochs metric             dataset   AL.iter              
0.000005 0.1   base   NaN       64  None      5      image_to_text_R@1  RSICD     NaN       3.287917   
                                                                        Sydney    NaN       2.523333   
                                                                        UCM       NaN       7.255833   
                                                     image_to_text_R@10 RSICD     NaN      24.362083   
                                                                        Sydney    NaN      16.364583   
...                                                                                              ...   
0.000500 0.1   ours   NaN       128 soft.text 35     text_to_image_R@5  RSICD     NaN       0.482500   
                                                                        Sydney    NaN       1.010000   
                                                                        UCM       NaN       2.142500   
                                                     zeroshot-val-top1  RSICD-CLS NaN       3.525000   
                                                                        UCM-CLS   NaN       4.812500   

                                                                                                     \
                                                                                                std   
lr       ratio method AL.epochs bs  PL        epochs metric             dataset   AL.iter             
0.000005 0.1   base   NaN       64  None      5      image_to_text_R@1  RSICD     NaN      0.300637   
                                                                        Sydney    NaN      0.216788   
                                                                        UCM       NaN      1.424429   
                                                     image_to_text_R@10 RSICD     NaN      0.652633   
                                                                        Sydney    NaN      0.744504   
...                                                                                             ...   
0.000500 0.1   ours   NaN       128 soft.text 35     text_to_image_R@5  RSICD     NaN      0.045000   
                                                                        Sydney    NaN      0.000000   
                                                                        UCM       NaN      0.005000   
                                                     zeroshot-val-top1  RSICD-CLS NaN      0.226495   
                                                                        UCM-CLS   NaN      0.211877   

                                                                                                 
                                                                                          count  
lr       ratio method AL.epochs bs  PL        epochs metric             dataset   AL.iter        
0.000005 0.1   base   NaN       64  None      5      image_to_text_R@1  RSICD     NaN        24  
                                                                        Sydney    NaN        24  
                                                                        UCM       NaN        24  
                                                     image_to_text_R@10 RSICD     NaN        24  
                                                                        Sydney    NaN        24  
...                                                                                         ...  
0.000500 0.1   ours   NaN       128 soft.text 35     text_to_image_R@5  RSICD     NaN         4  
                                                                        Sydney    NaN         4  
                                                                        UCM       NaN         4  
                   

In [7]:
df_grouped.to_csv('eval.csv')

In [8]:
df_results = df_grouped.reset_index()
df_results['AL.epochs'] = df_results['AL.epochs'].fillna(df_results['epochs'])
df_results = df_results.sort_values(['metric', 'dataset', ('value', 'mean')])
# df_results = df_results.groupby(['metric', 'dataset'])
# df_results = df_grouped.reset_index()
# df_results = df_results.sort_values(['metric', 'dataset', ('value', 'mean')])

# df_2 = df_2[(df_2['method'] == 'ours') & (df_2['ratio'] == 0.2) & (df_2['PL'] == 'ot.image')] # (df_2['metric'] == 'image_to_text_R@5') &
# df_2 = df_2[(df_2['epochs'] > 20)  & (df_2['bs'] == 64) & (df_2['lr']==5e-5)] # &(df_2['lr']==5e-5)
# df_2 = df_2[(df_2['lr']==5e-5) & (df_2['bs'] == 64) & (df_2['method'] == 'base')] # &(df_2['lr']==5e-5)
display(df_results)

lr ratio method AL.epochs   bs         PL epochs  \
                                                              
5720  0.00050   0.1   ours      35.0  128  hard.text     35   
5320  0.00050   0.1   ours       5.0   64  soft.text      5   
5420  0.00050   0.1   ours      30.0   64  soft.text     30   
6200  0.00050   0.1   ours      15.0  128  soft.text     15   
6260  0.00050   0.1   ours      30.0  128  soft.text     30   
...       ...   ...    ...       ...  ...        ...    ...   
4299  0.00005   0.5   base       5.0   64       None      5   
4419  0.00005   0.7   base      15.0   64       None     15   
4379  0.00005   0.7   base       5.0   64       None      5   
4339  0.00005   0.5   base      15.0   64       None     15   
4399  0.00005   0.7   base      10.0   64       None     10   

                 metric  dataset AL.iter    value                  
                                             mean       std count  
5720  image_to_text_R@1    RSICD     NaN   0.0450  0.051962     4  
5320  image_to_text_R@1    RSICD     NaN   0.0675  0.045000     4  
5420  image_to_text_R@1    RSICD     NaN   0.0675  0.045000     4  
6200  image_to_text_R@1    RSICD     NaN   0.0675  0.045000     4  
6260  image_to_text_R@1    RSICD     NaN   0.0675  0.045000     4  
...                 ...      ...     ...      ...       ...   ...  
4299  zeroshot-val-top1  UCM-CLS     NaN  77.7550  3.436680     4  
4419  zeroshot-val-top1  UCM-CLS     NaN  77.9725  3.209884     4  
4379  zeroshot-val-top1  UCM-CLS     NaN  78.1875  5.256433     4  
4339  zeroshot-val-top1  UCM-CLS     NaN  78.2875  3.939225     4  
4399  zeroshot-val-top1  UCM-CLS     NaN  78.3975  3.115032     4  

[6300 rows x 13 columns]

In [9]:
df_results.shape

(6300, 13)

In [10]:
# Results with active learning
full_display(df_results[(df_results['AL.iter']>0) & (df_results['epochs']==15)]) #.groupby(['metric', 'dataset']).tail(3)

lr ratio method AL.epochs  bs    PL epochs              metric  \
                                                                            
2140  0.00005   0.1   base      15.0  64  None     15   image_to_text_R@1   
2280  0.00005   0.1   base      15.0  64  None     15   image_to_text_R@1   
2282  0.00005   0.1   base      15.0  64  None     15   image_to_text_R@1   
2281  0.00005   0.1   base      15.0  64  None     15   image_to_text_R@1   
4180  0.00005   0.3   base      15.0  64  None     15   image_to_text_R@1   
4260  0.00005   0.5   base      15.0  64  None     15   image_to_text_R@1   
4340  0.00005   0.7   base      15.0  64  None     15   image_to_text_R@1   
2285  0.00005   0.1   base      15.0  64  None     15   image_to_text_R@1   
2141  0.00005   0.1   base      15.0  64  None     15   image_to_text_R@1   
2286  0.00005   0.1   base      15.0  64  None     15   image_to_text_R@1   
2284  0.00005   0.1   base      15.0  64  None     15   image_to_text_R@1   
4181  0.00005   0.3   base      15.0  64  None     15   image_to_text_R@1   
4261  0.00005   0.5   base      15.0  64  None     15   image_to_text_R@1   
4341  0.00005   0.7   base      15.0  64  None     15   image_to_text_R@1   
2288  0.00005   0.1   base      15.0  64  None     15   image_to_text_R@1   
2289  0.00005   0.1   base      15.0  64  None     15   image_to_text_R@1   
2290  0.00005   0.1   base      15.0  64  None     15   image_to_text_R@1   
2142  0.00005   0.1   base      15.0  64  None     15   image_to_text_R@1   
4262  0.00005   0.5   base      15.0  64  None     15   image_to_text_R@1   
4182  0.00005   0.3   base      15.0  64  None     15   image_to_text_R@1   
4342  0.00005   0.7   base      15.0  64  None     15   image_to_text_R@1   
2292  0.00005   0.1   base      15.0  64  None     15  image_to_text_R@10   
2143  0.00005   0.1   base      15.0  64  None     15  image_to_text_R@10   
2293  0.00005   0.1   base      15.0  64  None     15  image_to_text_R@10   
2294  0.00005   0.1   base      15.0  64  None     15  image_to_text_R@10   
4183  0.00005   0.3   base      15.0  64  None     15  image_to_text_R@10   
4263  0.00005   0.5   base      15.0  64  None     15  image_to_text_R@10   
4343  0.00005   0.7   base      15.0  64  None     15  image_to_text_R@10   
2297  0.00005   0.1   base      15.0  64  None     15  image_to_text_R@10   
2144  0.00005   0.1   base      15.0  64  None     15  image_to_text_R@10   
2298  0.00005   0.1   base      15.0  64  None     15  image_to_text_R@10   
2296  0.00005   0.1   base      15.0  64  None     15  image_to_text_R@10   
4184  0.00005   0.3   base      15.0  64  None     15  image_to_text_R@10   
4264  0.00005   0.5   base      15.0  64  None     15  image_to_text_R@10   
4344  0.00005   0.7   base      15.0  64  None     15  image_to_text_R@10   
2300  0.00005   0.1   base      15.0  64  None     15  image_to_text_R@10   
2301  0.00005   0.1   base      15.0  64  None     15  image_to_text_R@10   
2145  0.00005   0.1   base      15.0  64  None     15  image_to_text_R@10   
2302  0.00005   0.1   base      15.0  64  None     15  image_to_text_R@10   
4185  0.00005   0.3   base      15.0  64  None     15  image_to_text_R@10   
4265  0.00005   0.5   base      15.0  64  None     15  image_to_text_R@10   
4345  0.00005   0.7   base      15.0  64  None     15  image_to_text_R@10   
2304  0.00005   0.1   base      15.0  64  None     15   image_to_text_R@5   
2146  0.00005   0.1   base      15.0  64  None     15   image_to_text_R@5   
2306  0.00005   0.1   base      15.0  64  None     15   image_to_text_R@5   
2305  0.00005   0.1   base      15.0  64  None     15   image_to_text_R@5   
4186  0.00005   0.3   base      15.0  64  None     15   image_to_text_R@5   
4266  0.00005   0.5   base      15.0  64  None     15   image_to_text_R@5   
4346  0.00005   0.7   base      15.0  64  None     15   image_to_text_R@5   
2309  0.00005   0.1   base      15.0  64  None     15   image_to_text_R@5   
2147  0

In [11]:
# Results for base CLIP model (filtered a bit)
full_display(df_results[(df_results['AL.iter'].isna()) & (df_results['method'] == 'base') & (df_results['lr']==5e-5) & (df_results['ratio']==0.1)].groupby(['metric', 'dataset']).tail(3))

lr ratio method AL.epochs  bs    PL epochs              metric  \
                                                                            
2440  0.00005   0.1   base      25.0  64  None     25   image_to_text_R@1   
2480  0.00005   0.1   base      35.0  64  None     35   image_to_text_R@1   
2460  0.00005   0.1   base      30.0  64  None     30   image_to_text_R@1   
2441  0.00005   0.1   base      25.0  64  None     25   image_to_text_R@1   
2481  0.00005   0.1   base      35.0  64  None     35   image_to_text_R@1   
2461  0.00005   0.1   base      30.0  64  None     30   image_to_text_R@1   
2371  0.00005   0.1   base      20.0  64  None     20   image_to_text_R@1   
2482  0.00005   0.1   base      35.0  64  None     35   image_to_text_R@1   
2462  0.00005   0.1   base      30.0  64  None     30   image_to_text_R@1   
2463  0.00005   0.1   base      30.0  64  None     30  image_to_text_R@10   
2443  0.00005   0.1   base      25.0  64  None     25  image_to_text_R@10   
2483  0.00005   0.1   base      35.0  64  None     35  image_to_text_R@10   
2444  0.00005   0.1   base      25.0  64  None     25  image_to_text_R@10   
2464  0.00005   0.1   base      30.0  64  None     30  image_to_text_R@10   
2484  0.00005   0.1   base      35.0  64  None     35  image_to_text_R@10   
2465  0.00005   0.1   base      30.0  64  None     30  image_to_text_R@10   
2445  0.00005   0.1   base      25.0  64  None     25  image_to_text_R@10   
2485  0.00005   0.1   base      35.0  64  None     35  image_to_text_R@10   
2446  0.00005   0.1   base      25.0  64  None     25   image_to_text_R@5   
2466  0.00005   0.1   base      30.0  64  None     30   image_to_text_R@5   
2486  0.00005   0.1   base      35.0  64  None     35   image_to_text_R@5   
2487  0.00005   0.1   base      35.0  64  None     35   image_to_text_R@5   
2447  0.00005   0.1   base      25.0  64  None     25   image_to_text_R@5   
2467  0.00005   0.1   base      30.0  64  None     30   image_to_text_R@5   
2395  0.00005   0.1   base      20.0  64  None     20   image_to_text_R@5   
2468  0.00005   0.1   base      30.0  64  None     30   image_to_text_R@5   
2488  0.00005   0.1   base      35.0  64  None     35   image_to_text_R@5   
2449  0.00005   0.1   base      25.0  64  None     25   text_to_image_R@1   
2469  0.00005   0.1   base      30.0  64  None     30   text_to_image_R@1   
2489  0.00005   0.1   base      35.0  64  None     35   text_to_image_R@1   
2403  0.00005   0.1   base      20.0  64  None     20   text_to_image_R@1   
2470  0.00005   0.1   base      30.0  64  None     30   text_to_image_R@1   
2490  0.00005   0.1   base      35.0  64  None     35   text_to_image_R@1   
2407  0.00005   0.1   base      20.0  64  None     20   text_to_image_R@1   
2491  0.00005   0.1   base      35.0  64  None     35   text_to_image_R@1   
2471  0.00005   0.1   base      30.0  64  None     30   text_to_image_R@1   
2452  0.00005   0.1   base      25.0  64  None     25  text_to_image_R@10   
2492  0.00005   0.1   base      35.0  64  None     35  text_to_image_R@10   
2472  0.00005   0.1   base      30.0  64  None     30  text_to_image_R@10   
2453  0.00005   0.1   base      25.0  64  None     25  text_to_image_R@10   
2493  0.00005   0.1   base      35.0  64  None     35  text_to_image_R@10   
2473  0.00005   0.1   base      30.0  64  None     30  text_to_image_R@10   
2454  0.00005   0.1   base      25.0  64  None     25  text_to_image_R@10   
2474  0.00005   0.1   base      30.0  64  None     30  text_to_image_R@10   
2494  0.00005   0.1   base      35.0  64  None     35  text_to_image_R@10   
2455  0.00005   0.1   base      25.0  64  None     25   text_to_image_R@5   
2495  0.00005   0.1   base      35.0  64  None     35   text_to_image_R@5   
2475  0.00005   0.1   base      30.0  64  None     30   text_to_image_R@5   
2476  0.00005   0.1   base      30.0  64  None     30   text_to_image_R@5   
2456  0.00005   0.1   base      25.0  64  None     25   text_to_image_R@5   
2496  0

In [12]:
# Results for S-CLIP model (filtered a bit)
# & (df_results['lr']==5e-5)
df_S_CLIP = df_results[(df_results['AL.iter'].isna()) & (df_results['method'] == 'ours') & (df_results['PL'].str.contains('ot.'))]
df_PL_soft = df_results[(df_results['AL.iter'].isna()) & (df_results['method'] == 'ours') & (df_results['PL'].str.contains('soft.'))]
df_PL_hard = df_results[(df_results['AL.iter'].isna()) & (df_results['method'] == 'ours') & (df_results['PL'].str.contains('hard.'))]

In [13]:
full_display(df_S_CLIP.groupby(['metric', 'dataset']).tail(3))

lr ratio method AL.epochs   bs        PL epochs  \
                                                             
3700  0.00005   0.1   ours      25.0  128  ot.image     25   
3720  0.00005   0.1   ours      30.0  128  ot.image     30   
3740  0.00005   0.1   ours      35.0  128  ot.image     35   
3661  0.00005   0.1   ours      15.0  128  ot.image     15   
3721  0.00005   0.1   ours      30.0  128  ot.image     30   
3741  0.00005   0.1   ours      35.0  128  ot.image     35   
3702  0.00005   0.1   ours      25.0  128  ot.image     25   
3722  0.00005   0.1   ours      30.0  128  ot.image     30   
3742  0.00005   0.1   ours      35.0  128  ot.image     35   
3703  0.00005   0.1   ours      25.0  128  ot.image     25   
3743  0.00005   0.1   ours      35.0  128  ot.image     35   
3723  0.00005   0.1   ours      30.0  128  ot.image     30   
3704  0.00005   0.1   ours      25.0  128  ot.image     25   
3724  0.00005   0.1   ours      30.0  128  ot.image     30   
3744  0.00005   0.1   ours      35.0  128  ot.image     35   
2905  0.00005   0.1   ours      35.0   64  ot.image     35   
3725  0.00005   0.1   ours      30.0  128  ot.image     30   
3745  0.00005   0.1   ours      35.0  128  ot.image     35   
3706  0.00005   0.1   ours      25.0  128  ot.image     25   
3726  0.00005   0.1   ours      30.0  128  ot.image     30   
3746  0.00005   0.1   ours      35.0  128  ot.image     35   
2887  0.00005   0.1   ours      30.0   64  ot.image     30   
3727  0.00005   0.1   ours      30.0  128  ot.image     30   
3747  0.00005   0.1   ours      35.0  128  ot.image     35   
3748  0.00005   0.1   ours      35.0  128  ot.image     35   
3728  0.00005   0.1   ours      30.0  128  ot.image     30   
3708  0.00005   0.1   ours      25.0  128  ot.image     25   
3729  0.00005   0.1   ours      30.0  128  ot.image     30   
3749  0.00005   0.1   ours      35.0  128  ot.image     35   
2909  0.00005   0.1   ours      35.0   64  ot.image     35   
3750  0.00005   0.1   ours      35.0  128  ot.image     35   
3730  0.00005   0.1   ours      30.0  128  ot.image     30   
3710  0.00005   0.1   ours      25.0  128  ot.image     25   
3711  0.00005   0.1   ours      25.0  128  ot.image     25   
3731  0.00005   0.1   ours      30.0  128  ot.image     30   
3751  0.00005   0.1   ours      35.0  128  ot.image     35   
3712  0.00005   0.1   ours      25.0  128  ot.image     25   
3732  0.00005   0.1   ours      30.0  128  ot.image     30   
3752  0.00005   0.1   ours      35.0  128  ot.image     35   
3713  0.00005   0.1   ours      25.0  128  ot.image     25   
3733  0.00005   0.1   ours      30.0  128  ot.image     30   
3753  0.00005   0.1   ours      35.0  128  ot.image     35   
3734  0.00005   0.1   ours      30.0  128  ot.image     30   
2914  0.00005   0.1   ours      35.0   64  ot.image     35   
3754  0.00005   0.1   ours      35.0  128  ot.image     35   
3715  0.00005   0.1   ours      25.0  128  ot.image     25   
3755  0.00005   0.1   ours      35.0  128  ot.image     35   
3735  0.00005   0.1   ours      30.0  128  ot.image     30   
3716  0.00005   0.1   ours      25.0  128  ot.image     25   
3736  0.00005   0.1   ours      30.0  128  ot.image     30   
3756  0.00005   0.1   ours      35.0  128  ot.image     35   
3717  0.00005   0.1   ours      25.0  128  ot.image     25   
3737  0.00005   0.1   ours      30.0  128  ot.image     30   
3757  0.00005   0.1   ours      35.0  128  ot.image     35   
3718  0.00005   0.1   ours      25.0  128  ot.image     25   
3738  0.00005   0.1   ours      30.0  128  ot.image     30   
3758  0.00005   0.1   ours      35.0  128  ot.image     35   
3719  0.00005   0.1   ours      25.0  128  ot.image     25   
3739  0.00005   0.1   ours      30.0  128  ot.image     30   
3759  0.00005   0.1   ours      35.0  128  ot.image     35   

                  metric    dataset AL.iter      value                  
                                                  mean       std count  
3700   image_to_text

In [14]:
full_display(df_PL_soft.groupby(['metric', 'dataset']).tail(3))

lr ratio method AL.epochs   bs          PL epochs  \
                                                               
3300  0.00005   0.1   ours      30.0   64   soft.text     30   
3320  0.00005   0.1   ours      35.0   64   soft.text     35   
4000  0.00005   0.1   ours      30.0  128  soft.image     30   
3981  0.00005   0.1   ours      25.0  128  soft.image     25   
3301  0.00005   0.1   ours      30.0   64   soft.text     30   
3321  0.00005   0.1   ours      35.0   64   soft.text     35   
3322  0.00005   0.1   ours      35.0   64   soft.text     35   
4122  0.00005   0.1   ours      25.0  128   soft.text     25   
4162  0.00005   0.1   ours      35.0  128   soft.text     35   
4163  0.00005   0.1   ours      35.0  128   soft.text     35   
3323  0.00005   0.1   ours      35.0   64   soft.text     35   
3303  0.00005   0.1   ours      30.0   64   soft.text     30   
4144  0.00005   0.1   ours      30.0  128   soft.text     30   
4164  0.00005   0.1   ours      35.0  128   soft.text     35   
3284  0.00005   0.1   ours      25.0   64   soft.text     25   
4005  0.00005   0.1   ours      30.0  128  soft.image     30   
3325  0.00005   0.1   ours      35.0   64   soft.text     35   
3305  0.00005   0.1   ours      30.0   64   soft.text     30   
4026  0.00005   0.1   ours      35.0  128  soft.image     35   
3306  0.00005   0.1   ours      30.0   64   soft.text     30   
3326  0.00005   0.1   ours      35.0   64   soft.text     35   
4127  0.00005   0.1   ours      25.0  128   soft.text     25   
4147  0.00005   0.1   ours      30.0  128   soft.text     30   
4167  0.00005   0.1   ours      35.0  128   soft.text     35   
3308  0.00005   0.1   ours      30.0   64   soft.text     30   
4148  0.00005   0.1   ours      30.0  128   soft.text     30   
3328  0.00005   0.1   ours      35.0   64   soft.text     35   
3329  0.00005   0.1   ours      35.0   64   soft.text     35   
3309  0.00005   0.1   ours      30.0   64   soft.text     30   
4149  0.00005   0.1   ours      30.0  128   soft.text     30   
4170  0.00005   0.1   ours      35.0  128   soft.text     35   
4150  0.00005   0.1   ours      30.0  128   soft.text     30   
4010  0.00005   0.1   ours      30.0  128  soft.image     30   
4111  0.00005   0.1   ours      20.0  128   soft.text     20   
4011  0.00005   0.1   ours      30.0  128  soft.image     30   
4031  0.00005   0.1   ours      35.0  128  soft.image     35   
3292  0.00005   0.1   ours      25.0   64   soft.text     25   
3312  0.00005   0.1   ours      30.0   64   soft.text     30   
3332  0.00005   0.1   ours      35.0   64   soft.text     35   
4153  0.00005   0.1   ours      30.0  128   soft.text     30   
4173  0.00005   0.1   ours      35.0  128   soft.text     35   
4033  0.00005   0.1   ours      35.0  128  soft.image     35   
4034  0.00005   0.1   ours      35.0  128  soft.image     35   
3314  0.00005   0.1   ours      30.0   64   soft.text     30   
3334  0.00005   0.1   ours      35.0   64   soft.text     35   
3295  0.00005   0.1   ours      25.0   64   soft.text     25   
3315  0.00005   0.1   ours      30.0   64   soft.text     30   
3335  0.00005   0.1   ours      35.0   64   soft.text     35   
3996  0.00005   0.1   ours      25.0  128  soft.image     25   
4036  0.00005   0.1   ours      35.0  128  soft.image     35   
4016  0.00005   0.1   ours      30.0  128  soft.image     30   
4017  0.00005   0.1   ours      30.0  128  soft.image     30   
4177  0.00005   0.1   ours      35.0  128   soft.text     35   
4157  0.00005   0.1   ours      30.0  128   soft.text     30   
4038  0.00005   0.1   ours      35.0  128  soft.image     35   
3318  0.00005   0.1   ours      30.0   64   soft.text     30   
3338  0.00005   0.1   ours      35.0   64   soft.text     35   
4159  0.00005   0.1   ours      30.0  128   soft.text     30   
4019  0.00005   0.1   ours      30.0  128  soft.image     30   
4039  0.00005   0.1   ours      35.0  128  soft.image     35   

                  metric    dataset AL.ite

In [15]:
full_display(df_PL_hard.groupby(['metric', 'dataset']).tail(3))

lr ratio method AL.epochs   bs          PL epochs  \
                                                               
2720  0.00005   0.1   ours      25.0   64   hard.text     25   
3600  0.00005   0.1   ours      35.0  128   hard.text     35   
2760  0.00005   0.1   ours      35.0   64   hard.text     35   
2721  0.00005   0.1   ours      25.0   64   hard.text     25   
3581  0.00005   0.1   ours      30.0  128   hard.text     30   
3601  0.00005   0.1   ours      35.0  128   hard.text     35   
3442  0.00005   0.1   ours      30.0  128  hard.image     30   
3462  0.00005   0.1   ours      35.0  128  hard.image     35   
3582  0.00005   0.1   ours      30.0  128   hard.text     30   
3603  0.00005   0.1   ours      35.0  128   hard.text     35   
2743  0.00005   0.1   ours      30.0   64   hard.text     30   
2763  0.00005   0.1   ours      35.0   64   hard.text     35   
2744  0.00005   0.1   ours      30.0   64   hard.text     30   
2764  0.00005   0.1   ours      35.0   64   hard.text     35   
3604  0.00005   0.1   ours      35.0  128   hard.text     35   
2605  0.00005   0.1   ours      30.0   64  hard.image     30   
2765  0.00005   0.1   ours      35.0   64   hard.text     35   
2745  0.00005   0.1   ours      30.0   64   hard.text     30   
2766  0.00005   0.1   ours      35.0   64   hard.text     35   
2726  0.00005   0.1   ours      25.0   64   hard.text     25   
2746  0.00005   0.1   ours      30.0   64   hard.text     30   
2727  0.00005   0.1   ours      25.0   64   hard.text     25   
3587  0.00005   0.1   ours      30.0  128   hard.text     30   
3607  0.00005   0.1   ours      35.0  128   hard.text     35   
3608  0.00005   0.1   ours      35.0  128   hard.text     35   
2628  0.00005   0.1   ours      35.0   64  hard.image     35   
3588  0.00005   0.1   ours      30.0  128   hard.text     30   
2729  0.00005   0.1   ours      25.0   64   hard.text     25   
2749  0.00005   0.1   ours      30.0   64   hard.text     30   
2769  0.00005   0.1   ours      35.0   64   hard.text     35   
3570  0.00005   0.1   ours      25.0  128   hard.text     25   
3610  0.00005   0.1   ours      35.0  128   hard.text     35   
3590  0.00005   0.1   ours      30.0  128   hard.text     30   
2751  0.00005   0.1   ours      30.0   64   hard.text     30   
3611  0.00005   0.1   ours      35.0  128   hard.text     35   
2631  0.00005   0.1   ours      35.0   64  hard.image     35   
2732  0.00005   0.1   ours      25.0   64   hard.text     25   
2752  0.00005   0.1   ours      30.0   64   hard.text     30   
2772  0.00005   0.1   ours      35.0   64   hard.text     35   
2753  0.00005   0.1   ours      30.0   64   hard.text     30   
3613  0.00005   0.1   ours      35.0  128   hard.text     35   
3593  0.00005   0.1   ours      30.0  128   hard.text     30   
2754  0.00005   0.1   ours      30.0   64   hard.text     30   
2734  0.00005   0.1   ours      25.0   64   hard.text     25   
2774  0.00005   0.1   ours      35.0   64   hard.text     35   
2735  0.00005   0.1   ours      25.0   64   hard.text     25   
2755  0.00005   0.1   ours      30.0   64   hard.text     30   
2775  0.00005   0.1   ours      35.0   64   hard.text     35   
2736  0.00005   0.1   ours      25.0   64   hard.text     25   
3596  0.00005   0.1   ours      30.0  128   hard.text     30   
3616  0.00005   0.1   ours      35.0  128   hard.text     35   
2737  0.00005   0.1   ours      25.0   64   hard.text     25   
2777  0.00005   0.1   ours      35.0   64   hard.text     35   
2757  0.00005   0.1   ours      30.0   64   hard.text     30   
2738  0.00005   0.1   ours      25.0   64   hard.text     25   
2758  0.00005   0.1   ours      30.0   64   hard.text     30   
2778  0.00005   0.1   ours      35.0   64   hard.text     35   
2739  0.00005   0.1   ours      25.0   64   hard.text     25   
2759  0.00005   0.1   ours      30.0   64   hard.text     30   
2779  0.00005   0.1   ours      35.0   64   hard.text     35   

                  metric    dataset AL.ite